Aby zrealizować to zadanie, postępujemy według następujących kroków:

### Krok 1: Przygotowanie zbioru danych
1. **Pobranie i załadowanie danych**:
   - Ściągamy dane z [podanego źródła UCI Wine Dataset](http://archive.ics.uci.edu/dataset/109/wine).
   - Ładujemy dane z pliku CSV przy pomocy bibliotek takich jak `pandas` lub `numpy`.

2. **Przetasowanie zbioru**:
   - Przetasujemy dane, aby zapewnić losowe rozmieszczenie przykładów z każdej z 3 kategorii, co jest istotne dla poprawnego trenowania i walidacji modelu.

3. **Kodowanie kategorii**:
   - Kategoria wina zostanie zakodowana w formie "one-hot encoding" przy użyciu funkcji `np.eye()` lub narzędzia z biblioteki `pandas` – `pd.get_dummies()`, aby każda kategoria była reprezentowana w formie wektora (np. `[1,0,0]`, `[0,1,0]`, `[0,0,1]`).

### Krok 2: Przygotowanie modelu i funkcji celu
1. **Funkcja celu i aktywacja wyjścia**:
   - Ponieważ mamy do czynienia z klasyfikacją wieloklasową, najlepszą funkcją celu jest `categorical_crossentropy`, a funkcją aktywacji na wyjściu – `softmax`, która konwertuje wynik na prawdopodobieństwa przynależności do danej kategorii.

2. **Tworzenie modeli w Kerasie**:
   - **Model 1**: Sequential z podstawową liczbą warstw.
   - **Model 2**: Sequential z bardziej zaawansowaną konfiguracją (np. inną liczbą warstw, funkcjami aktywacji i metodami inicjalizacji wag).
   - W obu modelach podajemy dodatkowe informacje, takie jak nazwy warstw.

### Krok 3: Trening i testowanie modelu
1. **Podział zbioru danych**:
   - Podzielimy zbiór na część treningową i testową (np. 80% do treningu, 20% do testów).

2. **Parametry treningu**:
   - Określimy `learning_rate`, `batch_size`, oraz liczbę `epok`, które będą spójne między modelami.

3. **Uczenie i analiza wyników w TensorBoard**:
   - Podczas trenowania modeli, uruchomimy TensorBoard, aby śledzić krzywe uczenia (loss, accuracy) oraz obserwować konwergencję obu modeli.

4. **Wyniki i wnioski**:
   - Porównamy oba modele na podstawie wyników (dokładności, przebiegu krzywych uczenia).
   - Wyciągniemy wnioski, który model lepiej się sprawdził i jakie mogą być przyczyny tej różnicy (np. nadmierne dopasowanie, stabilność konwergencji, struktura sieci).

### Krok 4: Przygotowanie programu interaktywnego
Po wytrenowaniu najlepszego modelu przygotujemy prosty interfejs, który:
1. Pozwala użytkownikowi wprowadzić wartości cech wina (atrybuty z zestawu danych).
2. Przetwarza wejście przez model, aby uzyskać kategorię wina w postaci liczby (np. 1, 2, lub 3).
3. Przykład kodu dla użytkownika:
   ```python
   # User inputs for wine characteristics
   wine_characteristics = np.array([[13.2, 2.77, 2.51, 18.5, 98, 2.64, 2.43, 0.26, 1.63, 5.4, 0.94, 3.17, 680]])
   prediction = model.predict(wine_characteristics)
   predicted_class = np.argmax(prediction) + 1
   print(f"Predicted wine category: {predicted_class}")
   ```

### Dokumentacja i opis wyników
1. **Opis modeli**:
   - Krótki opis struktury, liczby warstw i użytych funkcji aktywacji.
   
2. **Wyniki i analiza**:
   - Dołączamy krzywe uczenia i dokładność modeli.
   - Krótka interpretacja wyników – który model lepiej sobie radzi i dlaczego.

3. **Wnioski końcowe**:
   - Opisujemy, jaki model był bardziej skuteczny i potencjalne przyczyny tego efektu (np. głębsza sieć może być bardziej podatna na przetrenowanie, a płytka może nie oddać pełnej złożoności danych).

Pamiętaj, aby zapisać projekt jako notatnik lub plik Python, wraz z dokumentacją krzywych uczenia i opisem wyników.

Oto kompletny kod Python, który realizuje Twoje zadanie krok po kroku:

Krok 1: Przygotowanie zbioru danych

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import TensorBoard
import datetime

# Pobranie danych
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data"
columns = ['Class', 'Alcohol', 'Malic_acid', 'Ash', 'Alcalinity_of_ash', 'Magnesium', 
           'Total_phenols', 'Flavanoids', 'Nonflavanoid_phenols', 'Proanthocyanins', 
           'Color_intensity', 'Hue', 'OD280/OD315', 'Proline']
data = pd.read_csv(url, header=None, names=columns)

# Przetasowanie danych
data = data.sample(frac=1).reset_index(drop=True)

# Oddzielenie cech od etykiet
X = data.drop('Class', axis=1).values
y = data['Class'].values.reshape(-1, 1)

# One-hot encoding etykiet
encoder = OneHotEncoder(sparse=False)
y_encoded = encoder.fit_transform(y)

# Podział na zbiór treningowy i testowy
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)


Krok 2: Definicja modeli
Poniżej znajdują się dwie różne architektury modelu.

Model 1

In [ ]:
# Model 1 - Prosta architektura
model_1 = Sequential(name="Model_1")
model_1.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],), name="Hidden_Layer_1"))
model_1.add(Dense(32, activation='relu', name="Hidden_Layer_2"))
model_1.add(Dense(3, activation='softmax', name="Output_Layer"))

# Kompilacja modelu
model_1.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                loss='categorical_crossentropy', 
                metrics=['accuracy'])


Model 2

In [ ]:
# Model 2 - Rozszerzona architektura
model_2 = Sequential(name="Model_2")
model_2.add(Dense(128, activation='relu', input_shape=(X_train.shape[1],), name="Hidden_Layer_1"))
model_2.add(Dense(64, activation='relu', name="Hidden_Layer_2"))
model_2.add(Dense(32, activation='relu', name="Hidden_Layer_3"))
model_2.add(Dense(3, activation='softmax', name="Output_Layer"))

# Kompilacja modelu
model_2.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                loss='categorical_crossentropy', 
                metrics=['accuracy'])


Krok 3: Trening modeli i TensorBoard

In [ ]:
# Ustawienia TensorBoard
log_dir_1 = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + "_model_1"
tensorboard_callback_1 = TensorBoard(log_dir=log_dir_1, histogram_freq=1)

log_dir_2 = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + "_model_2"
tensorboard_callback_2 = TensorBoard(log_dir=log_dir_2, histogram_freq=1)

# Trening modelu 1
history_1 = model_1.fit(X_train, y_train, 
                        epochs=100, 
                        batch_size=32, 
                        validation_data=(X_test, y_test), 
                        callbacks=[tensorboard_callback_1])

# Trening modelu 2
history_2 = model_2.fit(X_train, y_train, 
                        epochs=100, 
                        batch_size=32, 
                        validation_data=(X_test, y_test), 
                        callbacks=[tensorboard_callback_2])


Krok 4: Ocena modeli i wybór najlepszego

In [ ]:
# Ocena modelu 1
test_loss_1, test_acc_1 = model_1.evaluate(X_test, y_test, verbose=0)
print(f"Model 1 - Test accuracy: {test_acc_1:.4f}")

# Ocena modelu 2
test_loss_2, test_acc_2 = model_2.evaluate(X_test, y_test, verbose=0)
print(f"Model 2 - Test accuracy: {test_acc_2:.4f}")

# Wybór najlepszego modelu
best_model = model_1 if test_acc_1 > test_acc_2 else model_2


Krok 5: Interaktywny model predykcji dla użytkownika

In [ ]:
# Funkcja predykcyjna dla użytkownika
def predict_wine_category(features):
    prediction = best_model.predict(np.array([features]))
    predicted_class = np.argmax(prediction) + 1
    return predicted_class

# Przykład użycia
user_input = np.array([13.2, 2.77, 2.51, 18.5, 98, 2.64, 2.43, 0.26, 1.63, 5.4, 0.94, 3.17, 680])
print(f"Predicted wine category: {predict_wine_category(user_input)}")


Krok 6: Analiza wyników i TensorBoard
W celu analizy wyników należy uruchomić TensorBoard:

In [ ]:
tensorboard --logdir=logs/fit


Oto kompletny kod Python, który realizuje Twoje zadanie krok po kroku:

### Krok 1: Przygotowanie zbioru danych

```python
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import TensorBoard
import datetime

# Pobranie danych
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data"
columns = ['Class', 'Alcohol', 'Malic_acid', 'Ash', 'Alcalinity_of_ash', 'Magnesium', 
           'Total_phenols', 'Flavanoids', 'Nonflavanoid_phenols', 'Proanthocyanins', 
           'Color_intensity', 'Hue', 'OD280/OD315', 'Proline']
data = pd.read_csv(url, header=None, names=columns)

# Przetasowanie danych
data = data.sample(frac=1).reset_index(drop=True)

# Oddzielenie cech od etykiet
X = data.drop('Class', axis=1).values
y = data['Class'].values.reshape(-1, 1)

# One-hot encoding etykiet
encoder = OneHotEncoder(sparse=False)
y_encoded = encoder.fit_transform(y)

# Podział na zbiór treningowy i testowy
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)
```

### Krok 2: Definicja modeli

Poniżej znajdują się dwie różne architektury modelu.

#### Model 1

```python
# Model 1 - Prosta architektura
model_1 = Sequential(name="Model_1")
model_1.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],), name="Hidden_Layer_1"))
model_1.add(Dense(32, activation='relu', name="Hidden_Layer_2"))
model_1.add(Dense(3, activation='softmax', name="Output_Layer"))

# Kompilacja modelu
model_1.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                loss='categorical_crossentropy', 
                metrics=['accuracy'])
```

#### Model 2

```python
# Model 2 - Rozszerzona architektura
model_2 = Sequential(name="Model_2")
model_2.add(Dense(128, activation='relu', input_shape=(X_train.shape[1],), name="Hidden_Layer_1"))
model_2.add(Dense(64, activation='relu', name="Hidden_Layer_2"))
model_2.add(Dense(32, activation='relu', name="Hidden_Layer_3"))
model_2.add(Dense(3, activation='softmax', name="Output_Layer"))

# Kompilacja modelu
model_2.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                loss='categorical_crossentropy', 
                metrics=['accuracy'])
```

### Krok 3: Trening modeli i TensorBoard

```python
# Ustawienia TensorBoard
log_dir_1 = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + "_model_1"
tensorboard_callback_1 = TensorBoard(log_dir=log_dir_1, histogram_freq=1)

log_dir_2 = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + "_model_2"
tensorboard_callback_2 = TensorBoard(log_dir=log_dir_2, histogram_freq=1)

# Trening modelu 1
history_1 = model_1.fit(X_train, y_train, 
                        epochs=100, 
                        batch_size=32, 
                        validation_data=(X_test, y_test), 
                        callbacks=[tensorboard_callback_1])

# Trening modelu 2
history_2 = model_2.fit(X_train, y_train, 
                        epochs=100, 
                        batch_size=32, 
                        validation_data=(X_test, y_test), 
                        callbacks=[tensorboard_callback_2])
```

### Krok 4: Ocena modeli i wybór najlepszego

```python
# Ocena modelu 1
test_loss_1, test_acc_1 = model_1.evaluate(X_test, y_test, verbose=0)
print(f"Model 1 - Test accuracy: {test_acc_1:.4f}")

# Ocena modelu 2
test_loss_2, test_acc_2 = model_2.evaluate(X_test, y_test, verbose=0)
print(f"Model 2 - Test accuracy: {test_acc_2:.4f}")

# Wybór najlepszego modelu
best_model = model_1 if test_acc_1 > test_acc_2 else model_2
```

### Krok 5: Interaktywny model predykcji dla użytkownika

```python
# Funkcja predykcyjna dla użytkownika
def predict_wine_category(features):
    prediction = best_model.predict(np.array([features]))
    predicted_class = np.argmax(prediction) + 1
    return predicted_class

# Przykład użycia
user_input = np.array([13.2, 2.77, 2.51, 18.5, 98, 2.64, 2.43, 0.26, 1.63, 5.4, 0.94, 3.17, 680])
print(f"Predicted wine category: {predict_wine_category(user_input)}")
```

### Krok 6: Analiza wyników i TensorBoard

W celu analizy wyników należy uruchomić TensorBoard:
```bash
tensorboard --logdir=logs/fit
```
Następnie można przejść do przeglądarki i śledzić krzywe uczenia dla obu modeli, aby lepiej zrozumieć ich zachowanie podczas treningu.

### Podsumowanie

1. **Krzywe uczenia i dokładność**: Obserwuj krzywe uczenia obu modeli na TensorBoard, aby zobaczyć, jak zmienia się `accuracy` i `loss`.
2. **Opis modeli**: Wybierz model z najlepszym wynikiem na zbiorze testowym i zwróć uwagę, czy liczba warstw oraz rozmiar warstw wpłynęły na ich skuteczność.
3. **Predykcja**: Przygotowany interfejs predykcyjny pozwala na sprawdzenie kategorii wina na podstawie wprowadzonych cech.

To kompletny kod do załadowania, przetworzenia, treningu oraz oceny modeli klasyfikacyjnych na zbiorze danych "Wine" – zgodnie z wymaganiami zadania.